# Imports

In [35]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid', {'legend.frameon':True})

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor

from IPython.display import clear_output

# Loads

In [36]:
df = pd.read_csv('csv/jobs_it_process.csv')
full = pd.DataFrame(df)
df = df.dropna(subset=['salary_mean', 'region'])
df = pd.concat([df, pd.get_dummies(df.pop('salary_origin_mode'), prefix='mode')], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1532 entries, 5 to 10351
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   _id                          1532 non-null   object 
 1   query                        1532 non-null   object 
 2   title                        1532 non-null   object 
 3   company                      1532 non-null   object 
 4   rating_mean                  1532 non-null   float64
 5   location                     1532 non-null   object 
 6   salary                       1532 non-null   object 
 7   summary                      1532 non-null   object 
 8   sponso                       1532 non-null   int64  
 9   contract                     1279 non-null   object 
 10  desc                         1532 non-null   object 
 11  rating_count                 1532 non-null   int64  
 12  day_since                    1532 non-null   object 
 13  salary_min       

In [37]:
df['region'].unique()

array(['île-de-france', 'grand est', 'nouvelle-aquitaine', 'occitanie',
       "provence-alpes-côte d'azur", 'auvergne-rhône-alpes',
       'hauts-de-france', 'bretagne', 'centre-val de loire',
       'bourgogne-franche-comté', 'normandie', 'pays de la loire',
       'corse'], dtype=object)

In [38]:
f_rating=['rating_mean', 'rating_count']
f_contract=list(df.columns[21:31])
f_mode=list(df.columns[35:])
f_query=list(df.columns[17:21])

# Explore

In [14]:
sample = pd.concat([df[df['dep'] == '33'],
    df[df['dep'] == '17'], df[df['dep'] == '16'], 
    df[df['dep'] == '24'], df[df['dep'] == '47'],
    df[df['dep'] == '40']])
print('total {}'.format(len(sample)))
sample['dep'].value_counts()

total 101


33    88
17     8
16     2
47     2
24     1
Name: dep, dtype: int64

# Paris

In [74]:
sample = df[df['region']=='île-de-france'][['salary_min', 'salary_max', 'salary_mean']+f_rating+f_contract+f_mode+f_query]
sample.info()
sample.to_csv('csv/paris.csv', index=False, encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 523 entries, 5 to 5743
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   salary_min                   523 non-null    float64
 1   salary_max                   523 non-null    float64
 2   salary_mean                  523 non-null    float64
 3   rating_mean                  523 non-null    float64
 4   rating_count                 523 non-null    int64  
 5   contract_contrat pro         523 non-null    int64  
 6   contract_intérim             523 non-null    int64  
 7   contract_temps plein         523 non-null    int64  
 8   contract_indépendant         523 non-null    int64  
 9   contract_cdd                 523 non-null    int64  
 10  contract_temps partiel       523 non-null    int64  
 11  contract_cdi                 523 non-null    int64  
 12  contract_stage               523 non-null    int64  
 13  contract_commission

In [75]:
sample = full[full['region']=='île-de-france'][['salary_min', 'salary_max', 'salary_mean']+f_rating+f_contract+f_query]
sample['mode_an'] = 1
sample.info()
sample.to_csv('csv/paris_unknow.csv', index=False, encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3580 entries, 0 to 5743
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   salary_min                   523 non-null    float64
 1   salary_max                   523 non-null    float64
 2   salary_mean                  523 non-null    float64
 3   rating_mean                  3580 non-null   float64
 4   rating_count                 3580 non-null   int64  
 5   contract_contrat pro         3580 non-null   int64  
 6   contract_intérim             3580 non-null   int64  
 7   contract_temps plein         3580 non-null   int64  
 8   contract_indépendant         3580 non-null   int64  
 9   contract_cdd                 3580 non-null   int64  
 10  contract_temps partiel       3580 non-null   int64  
 11  contract_cdi                 3580 non-null   int64  
 12  contract_stage               3580 non-null   int64  
 13  contract_commissio

# Lyon

In [76]:
sample = pd.concat([df[df['dep'] == '69'],
    df[df['dep'] == '71'], df[df['dep'] == '42'], 
    df[df['dep'] == '01'], df[df['dep'] == '38'],
    df[df['dep'] == '26'], df[df['dep'] == '07']])[['salary_min', 'salary_max', 'salary_mean']+f_rating+f_contract+f_mode+f_query]
sample.info()
sample.to_csv('csv/lyon.csv', index=False, encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 112 to 3384
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   salary_min                   159 non-null    float64
 1   salary_max                   159 non-null    float64
 2   salary_mean                  159 non-null    float64
 3   rating_mean                  159 non-null    float64
 4   rating_count                 159 non-null    int64  
 5   contract_contrat pro         159 non-null    int64  
 6   contract_intérim             159 non-null    int64  
 7   contract_temps plein         159 non-null    int64  
 8   contract_indépendant         159 non-null    int64  
 9   contract_cdd                 159 non-null    int64  
 10  contract_temps partiel       159 non-null    int64  
 11  contract_cdi                 159 non-null    int64  
 12  contract_stage               159 non-null    int64  
 13  contract_commissi

In [77]:
sample = pd.concat([full[full['dep'] == '69'],
    full[full['dep'] == '71'], full[full['dep'] == '42'], 
    full[full['dep'] == '01'], full[full['dep'] == '38'],
    full[full['dep'] == '26'], full[full['dep'] == '07']])[['salary_min', 'salary_max', 'salary_mean']+f_rating+f_contract+f_query]
sample['mode_an'] = 1
sample.info()
sample.to_csv('csv/lyon_unknow.csv', index=False, encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 975 entries, 15 to 3748
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   salary_min                   159 non-null    float64
 1   salary_max                   159 non-null    float64
 2   salary_mean                  159 non-null    float64
 3   rating_mean                  975 non-null    float64
 4   rating_count                 975 non-null    int64  
 5   contract_contrat pro         975 non-null    int64  
 6   contract_intérim             975 non-null    int64  
 7   contract_temps plein         975 non-null    int64  
 8   contract_indépendant         975 non-null    int64  
 9   contract_cdd                 975 non-null    int64  
 10  contract_temps partiel       975 non-null    int64  
 11  contract_cdi                 975 non-null    int64  
 12  contract_stage               975 non-null    int64  
 13  contract_commissio

# Toulouse

In [78]:
sample = pd.concat([df[df['dep'] == '31'],
    df[df['dep'] == '32'], df[df['dep'] == '82'], 
    df[df['dep'] == '81'], df[df['dep'] == '11'],
    df[df['dep'] == '09'], df[df['dep'] == '65']])[['salary_min', 'salary_max', 'salary_mean']+f_rating+f_contract+f_mode+f_query]
sample.info()
sample.to_csv('csv/toulouse.csv', index=False, encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 45 to 7540
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   salary_min                   77 non-null     float64
 1   salary_max                   77 non-null     float64
 2   salary_mean                  77 non-null     float64
 3   rating_mean                  77 non-null     float64
 4   rating_count                 77 non-null     int64  
 5   contract_contrat pro         77 non-null     int64  
 6   contract_intérim             77 non-null     int64  
 7   contract_temps plein         77 non-null     int64  
 8   contract_indépendant         77 non-null     int64  
 9   contract_cdd                 77 non-null     int64  
 10  contract_temps partiel       77 non-null     int64  
 11  contract_cdi                 77 non-null     int64  
 12  contract_stage               77 non-null     int64  
 13  contract_commission

In [79]:
sample = pd.concat([full[full['dep'] == '31'],
    full[full['dep'] == '32'], full[full['dep'] == '82'], 
    full[full['dep'] == '81'], full[full['dep'] == '11'],
    full[full['dep'] == '09'], full[full['dep'] == '65']])[['salary_min', 'salary_max', 'salary_mean']+f_rating+f_contract+f_query]
sample['mode_an'] = 1
sample.info()
sample.to_csv('csv/toulouse_unknow.csv', index=False, encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 602 entries, 45 to 7631
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   salary_min                   77 non-null     float64
 1   salary_max                   77 non-null     float64
 2   salary_mean                  77 non-null     float64
 3   rating_mean                  602 non-null    float64
 4   rating_count                 602 non-null    int64  
 5   contract_contrat pro         602 non-null    int64  
 6   contract_intérim             602 non-null    int64  
 7   contract_temps plein         602 non-null    int64  
 8   contract_indépendant         602 non-null    int64  
 9   contract_cdd                 602 non-null    int64  
 10  contract_temps partiel       602 non-null    int64  
 11  contract_cdi                 602 non-null    int64  
 12  contract_stage               602 non-null    int64  
 13  contract_commissio

# Nantes

In [80]:
sample = pd.concat([df[df['dep'] == '44'],
    df[df['dep'] == '85'], df[df['dep'] == '49'], 
    df[df['dep'] == '72'], df[df['dep'] == '53'],
    df[df['dep'] == '35'], df[df['dep'] == '56']])[['salary_min', 'salary_max', 'salary_mean']+f_rating+f_contract+f_mode+f_query]
sample.info()
sample.to_csv('csv/nantes.csv', index=False, encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194 entries, 2608 to 8494
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   salary_min                   194 non-null    float64
 1   salary_max                   194 non-null    float64
 2   salary_mean                  194 non-null    float64
 3   rating_mean                  194 non-null    float64
 4   rating_count                 194 non-null    int64  
 5   contract_contrat pro         194 non-null    int64  
 6   contract_intérim             194 non-null    int64  
 7   contract_temps plein         194 non-null    int64  
 8   contract_indépendant         194 non-null    int64  
 9   contract_cdd                 194 non-null    int64  
 10  contract_temps partiel       194 non-null    int64  
 11  contract_cdi                 194 non-null    int64  
 12  contract_stage               194 non-null    int64  
 13  contract_commiss

In [81]:
sample = pd.concat([full[full['dep'] == '44'],
    full[full['dep'] == '85'], full[full['dep'] == '49'], 
    full[full['dep'] == '72'], full[full['dep'] == '53'],
    full[full['dep'] == '35'], full[full['dep'] == '56']])[['salary_min', 'salary_max', 'salary_mean']+f_rating+f_contract+f_query]
sample.info()
sample.to_csv('csv/nantes_unknow.csv', index=False, encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1184 entries, 111 to 8494
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   salary_min                   194 non-null    float64
 1   salary_max                   194 non-null    float64
 2   salary_mean                  194 non-null    float64
 3   rating_mean                  1184 non-null   float64
 4   rating_count                 1184 non-null   int64  
 5   contract_contrat pro         1184 non-null   int64  
 6   contract_intérim             1184 non-null   int64  
 7   contract_temps plein         1184 non-null   int64  
 8   contract_indépendant         1184 non-null   int64  
 9   contract_cdd                 1184 non-null   int64  
 10  contract_temps partiel       1184 non-null   int64  
 11  contract_cdi                 1184 non-null   int64  
 12  contract_stage               1184 non-null   int64  
 13  contract_commiss

# Bordeaux

In [82]:
sample = pd.concat([df[df['dep'] == '33'],
    df[df['dep'] == '17'], df[df['dep'] == '16'], 
    df[df['dep'] == '24'], df[df['dep'] == '47'],
    df[df['dep'] == '40']])[['salary_min', 'salary_max', 'salary_mean']+f_rating+f_contract+f_mode+f_query]
sample.info()
sample.to_csv('csv/bordeaux.csv', index=False, encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 16 to 5749
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   salary_min                   101 non-null    float64
 1   salary_max                   101 non-null    float64
 2   salary_mean                  101 non-null    float64
 3   rating_mean                  101 non-null    float64
 4   rating_count                 101 non-null    int64  
 5   contract_contrat pro         101 non-null    int64  
 6   contract_intérim             101 non-null    int64  
 7   contract_temps plein         101 non-null    int64  
 8   contract_indépendant         101 non-null    int64  
 9   contract_cdd                 101 non-null    int64  
 10  contract_temps partiel       101 non-null    int64  
 11  contract_cdi                 101 non-null    int64  
 12  contract_stage               101 non-null    int64  
 13  contract_commissio

In [83]:
sample = pd.concat([full[full['dep'] == '33'],
    full[full['dep'] == '17'], full[full['dep'] == '16'], 
    full[full['dep'] == '24'], full[full['dep'] == '47'],
    full[full['dep'] == '40']])[['salary_min', 'salary_max', 'salary_mean']+f_rating+f_contract+f_query]
sample.info()
sample.to_csv('csv/bordeaux_unknow.csv', index=False, encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 608 entries, 16 to 6052
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   salary_min                   101 non-null    float64
 1   salary_max                   101 non-null    float64
 2   salary_mean                  101 non-null    float64
 3   rating_mean                  608 non-null    float64
 4   rating_count                 608 non-null    int64  
 5   contract_contrat pro         608 non-null    int64  
 6   contract_intérim             608 non-null    int64  
 7   contract_temps plein         608 non-null    int64  
 8   contract_indépendant         608 non-null    int64  
 9   contract_cdd                 608 non-null    int64  
 10  contract_temps partiel       608 non-null    int64  
 11  contract_cdi                 608 non-null    int64  
 12  contract_stage               608 non-null    int64  
 13  contract_commissio